# Creating the template library

In [ ]:
%matplotlib qt
import hyperspy.api as hs #General hyperspy package
import pyxem as pxm #Electron diffraction tools based on hyperspy
import numpy as np #General numerical and matrix support
import matplotlib.pyplot as plt #Plotting tools
import matplotlib.colors as mcolors #Some plotting color tools
from matplotlib.cm import ScalarMappable
import diffpy #Electron diffraction tools
import requests

#Import path handling tool
from pathlib import Path

#Import indexation and plotting tools
from diffsims.generators.rotation_list_generators import get_beam_directions_grid
from diffsims.libraries.structure_library import StructureLibrary
from diffsims.generators.diffraction_generator import DiffractionGenerator
from diffsims.generators.library_generator import DiffractionLibraryGenerator
from pyxem.utils import indexation_utils as iutls
from pyxem.utils import plotting_utils as putls
from pyxem.utils import polar_transform_utils as ptutls
from pyxem.utils import expt_utils as eutls

#Import orientation handling tools
from orix.quaternion import Rotation, symmetry, Orientation
from orix.vector.vector3d import Vector3d
from orix.projections import StereographicProjection
from orix import plot

import pickle

## Creating the template library for LNMO
First the pre processed dataset is loaded.

In [ ]:
datapath = Path('./example_sample_dataset.zspy')

In [ ]:
print(f'Loading dataset {datapath.absolute()}')
processed_data = hs.load(str(datapath), lazy=True)
scale = 0.009451434347767504
processed_data

## Creating structure library

In [ ]:
resolution = 0.35 #The angular spacing between each orientation

euler_angles = get_beam_directions_grid('cubic', resolution, mesh='spherified_cube_edge') #Get a uniform sampling of euler angles
print(f'Simulated {len(euler_angles)} orientations')

#Use Orix to visualize the orientations
cs = symmetry.Oh #FCC symmetry
orientations = Orientation(Rotation.from_euler(np.deg2rad(euler_angles)), symmetry=cs)
orientations.scatter('ipf')
plt.gcf()

#Convert the Orix orientations back to euler angles to make sure we are using the "same" orientations that we verified in the plots
euler_angles = np.rad2deg(orientations.to_euler())

#Load the .cif file
structure = diffpy.structure.loadStructure("./LNMO.cif")

#Create a structure library
structure_library = StructureLibrary(['LNMO'], [structure], [euler_angles])

In [ ]:
minimum_intensity = 1E-20 
max_excitation_error = 0.37E-2

diff_gen = DiffractionGenerator(accelerating_voltage=200,
                                precession_angle=1, 
                                scattering_params='xtables',
                                shape_factor_model="linear",
                                minimum_intensity=minimum_intensity
                                )
lib_gen = DiffractionLibraryGenerator(diff_gen) #Create a diffraction library generator

calibration = processed_data.axes_manager[-1].scale #Get the calibration from the signal
reciprocal_radius = np.max(np.abs(processed_data.axes_manager[-1].axis)) #Get the maximum reciprocal radius to simulate
half_shape = np.min(processed_data.axes_manager.signal_shape)//2 #Get the half-shape of the patterns

library = lib_gen.get_diffraction_library(structure_library,
                                          calibration=calibration, 
                                          reciprocal_radius=reciprocal_radius, 
                                          half_shape=half_shape, 
                                          with_direct_beam=False, 
                                          max_excitation_error=max_excitation_error
                                         ) #Simulate diffraction patterns for your library

simulations = library['LNMO']['simulations'] #Get the simulations that were produced
simulated_patterns = hs.signals.Signal2D([simulation.get_diffraction_pattern(sigma=1, shape=processed_data.axes_manager.signal_shape) for simulation in simulations]) #Get the diffraction patterns from the simulated library and put them in a signal for visualisation
simulated_patterns.plot(norm='symlog') #Plot the simulated patterns for verification.

## Save the templates and dictionary



In [ ]:

with open("./library.pkl", 'wb') as fp:
    pickle.dump(library, fp)